<a href="https://colab.research.google.com/github/senjoyee/udemy_langchain_python/blob/main/langchain_python_bootcamp_4_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install -qU langchain-openai

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

In [3]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
llm=ChatOpenAI()

In [5]:
beginner_template="""You are a physics teacher who is really focused on
beginners and explaining complex concepts in simple to understand terms.
You assume no prior knowledge. Here is your question:\n{input}"""

In [6]:
expert_template="""You are a physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a PhD in Physics.
Here is your question:\n{input}"""

ROUTE PROMPT INFORMATION

In [7]:
prompt_infos = [
                {"name": "beginner physics",
                 "description": "Answers basic physics questions",
                 "template":beginner_template},
                {"name": "advanced physics",
                 "description": "Anwers advanced physics questions",
                 "template": expert_template}
                ]

In [8]:
destination_chains = {}
for p_info in prompt_infos:
  name=p_info["name"]
  prompt_template=p_info["template"]
  prompt=ChatPromptTemplate.from_template(template=prompt_template)
  chain=LLMChain(llm=llm,prompt=prompt)
  destination_chains[name]=chain

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
destination_chains

{'beginner physics': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a physics teacher who is really focused on\nbeginners and explaining complex concepts in simple to understand terms.\nYou assume no prior knowledge. Here is your question:\n{input}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7ab2c6e51db0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7ab2c6e534c0>, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'advanced physics': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a physics professor who explains physics topics\nto advanced audience members. You can assume anyone you answer has a PhD in Physics.\nHere is your question:\n{input}'))]), llm=ChatOpenAI(clie

In [10]:
default_prompt=ChatPromptTemplate.from_template("{input}")
default_chain=LLMChain(llm=llm,prompt=default_prompt)

In [11]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [12]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [13]:
destinations = [f'{p["name"]}:{p["description"]}' for p in prompt_infos]

In [14]:
destinations

['beginner physics:Answers basic physics questions',
 'advanced physics:Anwers advanced physics questions']

In [15]:
destinations_str="\n".join(destinations)

In [16]:
print(destinations_str)

beginner physics:Answers basic physics questions
advanced physics:Anwers advanced physics questions


In [17]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [18]:
router_template=MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

In [19]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner physics:Answers basic physics questions
advanced physics:Anwers advanced physics questions



In [20]:
router_prompt=PromptTemplate(template=router_template,
                             input_variables=["input"],
                             output_parser=RouterOutputParser())

In [21]:
from langchain.chains.router import MultiPromptChain

In [22]:
router_chain=LLMRouterChain.from_llm(llm,router_prompt)

In [23]:
chain=MultiPromptChain(router_chain=router_chain,
                       destination_chains=destination_chains,
                       default_chain=default_chain,verbose=True
                       )

In [24]:
chain.run("How do magnets work?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How do magnets work?'}
> Finished chain.


'Magnets work by creating a magnetic field around them. This magnetic field is produced by the movement of electrons within the atoms of the magnet. Each electron has a property called "spin," which creates a tiny magnetic field. When these spins align in the same direction, they create a larger, stronger magnetic field.\n\nWhen two magnets are brought close together, their magnetic fields interact. If the poles of the magnets are opposite (one is north and the other is south), they will attract each other. This is because opposite poles attract each other, similar to how positive and negative charges attract in electricity.\n\nOn the other hand, if the poles are the same (both north or both south), they will repel each other. This is because like poles repel each other, similar to how two positive or two negative charges repel in electricity.\n\nOverall, magnets work by creating magnetic fields through the movement of electrons, and these fields interact with each other to either attr